In [3]:
import numpy as np
import pandas as pd


def loadData():
    items = {
        'A':{'Alice':'5.0','Bob':'3.0','Cathy':'2.5','Dave':'2.0'},
        'B':{'Alice':'2.0','Bob':'2.5','Cathy':'5.0','Dave':'2.0'},
        'C':{'Alice':'2.5','Bob':'3.0','Cathy':'2.0','Dave':'2.5'},
        'D':{'Alice':'3.0','Bob':'4.0','Cathy':'2.5','Dave':'4.0'}
    }
    return items

In [4]:
item_data = loadData() 
print(item_data)

{'A': {'Alice': '5.0', 'Bob': '3.0', 'Cathy': '2.5', 'Dave': '2.0'}, 'B': {'Alice': '2.0', 'Bob': '2.5', 'Cathy': '5.0', 'Dave': '2.0'}, 'C': {'Alice': '2.5', 'Bob': '3.0', 'Cathy': '2.0', 'Dave': '2.5'}, 'D': {'Alice': '3.0', 'Bob': '4.0', 'Cathy': '2.5', 'Dave': '4.0'}}


In [5]:
item_data = loadData()

similarity_matrix = pd.DataFrame(
    np.identity(len(item_data)),#单位矩阵
    index = item_data.keys(),
    columns = item_data.keys()
)

print(similarity_matrix)

     A    B    C    D
A  1.0  0.0  0.0  0.0
B  0.0  1.0  0.0  0.0
C  0.0  0.0  1.0  0.0
D  0.0  0.0  0.0  1.0


In [6]:
for i1,user1 in item_data.items():
    print(i1)
    print(user1)

A
{'Alice': '5.0', 'Bob': '3.0', 'Cathy': '2.5', 'Dave': '2.0'}
B
{'Alice': '2.0', 'Bob': '2.5', 'Cathy': '5.0', 'Dave': '2.0'}
C
{'Alice': '2.5', 'Bob': '3.0', 'Cathy': '2.0', 'Dave': '2.5'}
D
{'Alice': '3.0', 'Bob': '4.0', 'Cathy': '2.5', 'Dave': '4.0'}


In [13]:
#计算相似度矩阵
for i1,user1 in item_data.items():
    for i2,user2 in item_data.items():
        if i1 ==i2:
            continue
        vec1,vec2 = [],[]
        for user,rating1 in user1.items():
            rating2 = user2.get(user,-1)
            if rating2 == -1:
                continue
            vec1.append(float(rating1))
            vec2.append(float(rating2))
        print(vec1)
        print(vec2)
        print(np.corrcoef(np.array(vec1),np.array(vec2),rowvar=0).shape)
        similarity_matrix.loc[i1,i2] = np.corrcoef(np.array(vec1),
                                                   np.array(vec2),
                                                   rowvar=0)[0][1]#np.corrcoef返回的是一个2*2的矩阵，我们只取[0][1]
print(similarity_matrix)      

[5.0, 3.0, 2.5, 2.0]
[2.0, 2.5, 5.0, 2.0]
(2, 2)
[5.0, 3.0, 2.5, 2.0]
[2.5, 3.0, 2.0, 2.5]
(2, 2)
[5.0, 3.0, 2.5, 2.0]
[3.0, 4.0, 2.5, 4.0]
(2, 2)
[2.0, 2.5, 5.0, 2.0]
[5.0, 3.0, 2.5, 2.0]
(2, 2)
[2.0, 2.5, 5.0, 2.0]
[2.5, 3.0, 2.0, 2.5]
(2, 2)
[2.0, 2.5, 5.0, 2.0]
[3.0, 4.0, 2.5, 4.0]
(2, 2)
[2.5, 3.0, 2.0, 2.5]
[5.0, 3.0, 2.5, 2.0]
(2, 2)
[2.5, 3.0, 2.0, 2.5]
[2.0, 2.5, 5.0, 2.0]
(2, 2)
[2.5, 3.0, 2.0, 2.5]
[3.0, 4.0, 2.5, 4.0]
(2, 2)
[3.0, 4.0, 2.5, 4.0]
[5.0, 3.0, 2.5, 2.0]
(2, 2)
[3.0, 4.0, 2.5, 4.0]
[2.0, 2.5, 5.0, 2.0]
(2, 2)
[3.0, 4.0, 2.5, 4.0]
[2.5, 3.0, 2.0, 2.5]
(2, 2)
          A         B         C         D
A  1.000000 -0.341983  0.155230 -0.316862
B -0.341983  1.000000 -0.710669 -0.715653
C  0.155230 -0.710669  1.000000  0.816497
D -0.316862 -0.715653  0.816497  1.000000


In [12]:
X = np.array([[1, 2], [3, 4], [5, 6]])
Y = np.array([[1, 2, 3], [4, 3, 2], [5, 6, 7]])
print(np.corrcoef(X))
#print(np.corrcoef(X,Y))


[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [14]:
target_user = 'Alice'
target_item = 'D'
num = 2

sim_items=[]
sim_items_list = similarity_matrix[target_item].sort_values(ascending=False).index.tolist()
print(sim_items_list)

for item in sim_items_list:
    if target_user in item_data[item]:
        sim_items.append(item)
    if len(sim_items) == num:
        break
print(sim_items)

['D', 'C', 'A', 'B']
['D', 'C']


In [30]:
temp = list(item_data[target_item].values())

target_user_mean_rating = sum(map(float, temp)) / len(temp)#D的平均评分
print(target_user_mean_rating)
weighted_scores = 0
corr_values_sum = 0

target_item = 'D'
for item in sim_items:
    corr_value = similarity_matrix.loc[target_item,item]#D和A的相似度
    user_mean_rating = np.mean(list(item_data[item].values()))#A的平均评分

    weighted_scores += corr_value * (float(item_data[item][target_user]) - user_mean_rating)#D的评分-A的平均评分
    corr_values_sum += corr_value 

target_item_pred = target_user_mean_rating + weighted_scores / corr_values_sum

print(f"{target_item}的推荐得分是：{target_item_pred}")

TypeError: 'int' object is not callable